# Incredible Deep Learning in immunooncology with Pytorch

This tutorial requires some pre-requisites: basic Python and preferably Machine Learning knowledge, experience with Anaconda and Jupyter Lab. We provide useful links to quickly learn those, if you are not yet familiar with them:

- A neat introduction to python - https://www.w3schools.com/python/default.asp

- Introduction to Anaconda - an essential package manager for Python - https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html

- Introduction to Jupyter Lab - an intergrated development environment for writing Python code and data analysis. Jupyter Lab is based on Jupyter Notebook philosophy and runs Jupyter Notebooks, you can learn about them here - https://www.dataquest.io/blog/jupyter-notebook-tutorial/. If you want a Jupyter Lab tutorial, here is a nice video explaining all the basics - https://www.youtube.com/watch?v=NXwP8pSOiB8.

A nice intro to Deep Learning
https://towardsdatascience.com/an-introduction-to-deep-learning-af63448c122c

PyTorch intro
https://github.com/yunjey/pytorch-tutorial

Building pMHCI classifier
http://dmnfarrell.github.io/bioinformatics/mhclearning

Building pMHCI classifier 2
https://blogs.rstudio.com/tensorflow/posts/2018-01-29-dl-for-cancer-immunotherapy/

A PyTorch tutorial
https://github.com/spro/practical-pytorch/blob/master/char-rnn-classification/char-rnn-classification.ipynb

## Plan

- MHC and cancer immunotherapy
- Quick intro to Machine Learning
- Predicting peptide-MHCI binders
- Further directions and ideas to explore

## MHC and cancer immunotherapy

![MHC I class complex](assets/mhci.jpg)

MHC stands for major histocompatibility complex. It is an ensemble of proteins on the cell surface that present peptides to TCR receptor and determines histocompatibility through this interaction. All human body cells that have nucleus usually present cytosolic peptides of their own origin, derived from the cell metabolism and defective ribosomal products. 

In a cell, protein molecules of the host's own phenotype or of other biologic entities are continually synthesized and degraded. During intracellular infection or cell transformation foreign or mutated proteins also degrade and get presented on sell surface within the peptide-binding groove of the MHC molecule. Interestingly enough, T-cells can detect peptides presented on as little as 0.1%-1% MHC complexes. 

These peptides can be studied in the laboratory with HTS peptidome sequencing techniques including mass-spectrometry and various biochemical methods. One way to perform it, is to acquire cell lines expressing single MHC allele or in vitro purified MHC complex. After addition of a mixture of peptides to the system, it should be purified and the binders should be sequenced. 

TCR is responsible for recognition of both MHC complex and peptide epitope presented. If a cell does not bear MHC on its surface it cannot be detected by cytotoxic T-cell. Some cancers develop this strategy to avoid immune system inspection and consequent immune response. However these cells normally must be eliminated by natural killers (NK) and NKT subsets of lymphocytes. Thus, immune system is able do kill both cells that present foreign peptide and cells that lose their MHC due to mutation or another event, probably in the process of cancer transformation. 

This mechanism is very efficient in terms of infection and cell malignisation control. However the same mechanism poses several medical challenges. It comes with no surprise that the recipient-donor pair matching for biological materials and organ transplantation is one of the most difficult problems. It stems from the frequent transplant rejections caused by antigen mismatch that causes host-versus-graft disease. Sometimes graft-versus-host disease follows the receipt of transplanted tissue from a genetically different person. For instance, in the case of bone marrow transplantation the graft's lymphocytes may attack body cells recognised as foreign.


Peptide binding to MHC molecules is the key selection step in the Antigen-presentation pathway. This is essential for T cell immune responses. The ‘epitope’ is the peptide-MHC combination shown in the image at right. Key residues in the MHC contact the peptide and these differ between alleles. The prediction of peptide binding to MHC molecules has been much studied. The problem is simpler for class-I molecules since the binding peptide length is less variable (usually 8-11 but commonly 9). Typically binding predictors are based on training models with experimental binding affinity measurements with known peptide sequences. This data is available from the IEDB for many human alleles. New peptides can then be predicted based on their position specific similarity to the training data.


### Adoptive T-cell immunotherapy

Adoptive T-cell therapy is a form of cancer immunotherapy that aims to isolate tumor infiltrating T-cells from the tumor in the patient, possibly genetically engineer them to be cancer-specific, grow them in great numbers and reintroduce them into the body to fight the cancer. In order to terminate cancer cells, the T-cell needs to be activated by being exposed to tumor peptides bound to MHCI (pMHCI). By analyzing the tumor genetics, relevant peptides can be identified and depending on the patients particular type of MHCI, we can predict which pMHCI are likely to be present in the tumor in the patient and thus which pMHCIs should be used to activate the T-cells.

From [here](https://blogs.rstudio.com/tensorflow/posts/2018-01-29-dl-for-cancer-immunotherapy/)

## Quick intro to Machine Learning

Machine learning is all about algorithms or "machines" that learn *something* from the input data that helps you solve your task. But what task are you solving? There are different types of them, which can be combined in a number of high-level descriptions. We will focus on the one type of tasks - a classification task. 

### Classification task

Classification task is type of task where the algorithm predicts the label or class based on unlabeled data. The algorithm has to be trained first on a set of known objects and their labels. For example, a set may contain the pictures of pets, where a picture of a dog is an object, and "dog" is its label. In our case, we have peptides as objects and labels 'True" or "False" if they bind or do not bind the MHC class I respectively.

The typical workflow has four steps: 

- input data - the object-label pairs for model training

- create model - the exact algorithm that will predict us binders or dogs in the first example

- choose loss function - a function to evaluate how well our algorithm predicts the label

- choose optimisation algorithm - an algorithm that "guides" our model so it learns how to predict the right label

#### Uploading the training data and one-hot encoding it

In [ ]:
import pandas as pd

# upload data here
train_data = pd.read_csv("data/train.csv.gz")
print("Dataset size:", len(train_data), "peptides")
train_data.head()

For educational purposes we will be training the model only on just one allele. For this reason we will filter the dataset.

In [ ]:
from collections import Counter

# Count alleles
print(Counter(train_data["mhc"]))

In [ ]:
%matplotlib inline
from collections import Counter

from matplotlib import pyplot as plt
import seaborn as sns

# Count alleles

plt.figure(figsize=(10,10))
sns.countplot(y=train_data["mhc"])

In [ ]:
# Filter out all non-human alleles
train_data = train_data.loc[train_data["mhc"].str.startswith("HLA"), ]

plt.figure(figsize=(10,10))
sns.countplot(y=train_data["mhc"])

In [ ]:
# Task: filter out non HLAA0201 alleles

# your code is here

print("Filtered dataset size:", len(train_data), "peptides")

plt.figure(figsize=(10,10))
sns.countplot(y=train_data["mhc"])

In [ ]:
# Task: filter out non 9-mers
# You will need https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.len.html

In [ ]:
# Find and drop duplicates
train_size_pre = len(train_data)
train_data.drop_duplicates(["mhc", "sequence"], inplace=True)
print("Dropped", train_size_pre - len(train_data), "duplicates")
print("Deduplicated dataset:", len(train_data), "peptides")

In [ ]:
# Task: follow the same pipeline for the testing data
# Suggestion: try not to look very often to the previous code

# load data
# print a number of peptides in the data
# plot a distribution of alleles
# filter out all non HLAA0201 alleles
# filter out all non 9-mers
# remove duplicates
# how many duplicates did you remove?

In [ ]:
# Find any common peptides and drop them as well

test_data = pd.read_csv("data/test.csv.gz")

pd.merge(train_data, test_data, "inner", ["mhc", "sequence"]) # and there is none!

In [ ]:
import numpy as np

# Convert IC50 values

print(train_data["meas"].max())
train_data.loc[train_data["meas"] > 50000, "meas"] = 50000
print(train_data["meas"].max())
train_data["meas"] = 1 - np.log(train_data["meas"]) / np.log(50000)

# Task: do the same for test data
test_data = train_data

And that's why you need to learn how to make functions :-)

In [ ]:
import torch

# Create one-hot encoding

# First, create a dictionary aminoacid -> feature vector (one-hot vector)
def create_dict(dataframe):
    d = { x:y for y,x in enumerate(sorted(set(dataframe["sequence"].str.cat()))) }
    for i, aa in enumerate(d):
        vec = torch.zeros((len(d), ))
        vec[i] = 1
        d[aa] = vec
    return d

onehot_dict = create_dict(train_data)
onehot_dict

In [ ]:
from tqdm import tqdm

def make_onehot(dataframe, feature_dict):
    max_peptide_size = dataframe["sequence"].str.len().max()
    res = torch.zeros((len(dataframe), max_peptide_size, len(feature_dict))) # 3d tensor (X, Y, Z)
    for i, pep in tqdm(enumerate(dataframe["sequence"]), total=len(dataframe["sequence"])):
        for j, aa in enumerate(pep):
            res[i][j] = feature_dict[aa]
    return res

X_train = make_onehot(train_data, onehot_dict)
# X_test = make_onehot(test_data, onehot_dict)
print(X_train.shape)
X_train

Common Machine Learning models require feature engineering, i.e., the manual selection or creation of features using statistics, mathematical operations, etc. The Neural Networks are user-friendly and can process all features without any engineering, although the inner representation of data remains a black box for the data scientist.

Due to this fact, it is difficult to track the influence of individual features on the overall result. 

### Classification model

Now as we have finished with the data preprocessing and have got a nice small dataset to train on, we need a model. A mathemathical model, called "classifier", is a model that takes input X and uses it to predict an output.

Generally speaking, deep learning is a machine learning method that takes in an input train data, and uses it to predict an output of test data.

We will use PyTorch - a powerful framework for Deep Learning that most researchers in the world love. There are other frameworks, such as TensorFlow that is more popular than PyTorch (or at least what people say...), but PyTorch is better in our opinion.

<img src="https://cdn-images-1.medium.com/max/720/0*4aHRjVXRKsyUhm2b" width="600" />


A neural network is composed of input, hidden, and output layers — all of which are composed of “nodes” or "computational units" (previously referred as "neurons"). Input layers take in a numerical representation of data (e.g. images with pixel specs), output layers output predictions, while hidden layers are correlated with most of the computation.


<img src="https://cdn-images-1.medium.com/max/720/0*C_nYHONjxjUVgIYh" width="600" />


<img src="https://cdn-images-1.medium.com/max/720/0*5kA2vn4UqmI4iKLM" width="600" />



In [ ]:
# Here is an example of the model above done in PyTorch

from torch import nn
from torch.nn import Sequential, Linear, Sigmoid

model = nn.Sequential(Linear(3, 4), # input layer -> hidden layer 1
                      Sigmoid(), 
                      Linear(4, 4), # hidden layer 1 -> hidden layer 2
                      Sigmoid(),
                      Linear(4, 1), # hidden layer 2 -> output layer
                      Sigmoid())

### Loss function

But how model will understand, how to use the input data to learn from it? 
Given a large dataset of input and output pairs, a deep learning algorithm will try to minimize the difference between its prediction and expected output. By doing this, it tries to learn the association/pattern between given inputs and outputs — this in turn allows a deep learning model to generalize to inputs that it hasn’t seen before.

We will minimize the Binary Cross-Entropy loss.

In order to use BCE loss with Pytorch you just need to execute the following code:

In [ ]:
from torch.nn import BCELoss

criterion = BCELoss()

### Optimisation algorithm

batch learning
picture with local minima
speed of optimization (learning rate)
nn.optim.Adam

In [ ]:
from torch.optim import Adam

opt = Adam(model.parameters())

# Predicting peptide-MHCI binders

And finally we can integrate everything we learned so far into the main training procedure.

Algorithm for training neural networks:

  1. Check: overfit one batch - both train and test to test the model and functions
  2. Overfit: make the best overfitting model as possible
  3. Regularise: try to add regularizers to increase the validation score

For details see this blogpost: http://karpathy.github.io/2019/04/25/recipe/

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

THRESHOLD = 0.426
N_EPOCHS = 10
BATCH_SIZE = 32

y = torch.from_numpy(train_data["meas"].values).reshape((-1,1)).float()
y[y >= THRESHOLD] = 1
y[y < THRESHOLD] = 0

dataset = TensorDataset(X_train.view((X_train.shape[0], -1)), y)
dl = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

model = nn.Sequential(Linear(300, 64),
                      Sigmoid(), 
                      Linear(64, 1),
                      Sigmoid())
criterion = BCELoss()
opt = Adam(model.parameters(), lr=0.01)

for i_epoch in range(N_EPOCHS):
    stats = {"loss": [], "acc": []}
    for X, y_true in dl:
        opt.zero_grad()
        
        y_pred = model(X)
        loss = criterion(y_pred, y_true)
        loss.backward()
        opt.step()
        
        y_pred[y_pred >= THRESHOLD] = 1
        y_pred[y_pred < THRESHOLD] = 0
        stats["loss"].append(loss.item())
        stats["acc"].append(y_true.long().eq(y_pred.long()).sum() / X.shape[0])
    
    print("Epoch:", i_epoch, "Loss:", np.mean(stats["loss"]), "Acc:", np.mean(stats["acc"]))

In [ ]:
def test(model, criterion, dl):
    model.eval()
    with torch.no_grad():
        stats = {"loss": [], "acc": []}
        for X, y_true in dl:
            y_pred = model(X)
            loss = criterion(y_pred, y_true)

            y_pred[y_pred >= THRESHOLD] = 1
            y_pred[y_pred < THRESHOLD] = 0
            stats["loss"].append(loss.item())
            stats["acc"].append(y_true.long().eq(y_pred.long()).sum() / X.shape[0])
    
        print("Test:", i_epoch, "Loss:", np.mean(stats["loss"]), "Acc:", np.mean(stats["acc"]))
            
    model.train()
    
test(model, criterion, dataset)

# Interesting ideas to explore

## Per-allele model

### 1. Find the best encoding

There are numerous ways to encode amino acids. The most notable are BLOSUM, Kidera and Athley factors. You can find some of them in our [repository here](https://github.com/vadimnazarov/kidera-atchley). 

### 2. Use embeddings

Word embedding is a vector representation of a word, encoding its meaning in some context. 

[Here](https://israelg99.github.io/2017-03-23-Word2Vec-Explained/) is a quick and good introduction to word embeddings. And [here](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html) you can find a little bit harder PyTorch tutorial on word embeddings with PyTorch code needed for infer your own representation of amino acids.

### 3. Use different training subroutines

Changing learning rate, balancing batches or make it a regression task instead of classification may help.

### 4. Use convolutional networks to extract binding motifs

### 5. Use recurrent neural networks

#### 5.1. Use RNNs with Attention and extract binding motifs

### 6. Think about scaling: create a smallest and fastest possible model for quick analysis of 10,000,000 peptides

### 7. Create a binder space: use autoencoders, variational autoencoders or generative adversarial networks

## Pan-allele model

Pan-allele models such as [NetMHCpan](http://www.cbs.dtu.dk/services/NetMHCpan/) are models that uses information about both peptides and alleles. A model analyses both peptide and MHC allele sequences and can be used for prediction of previously unknown MHC alleles. Additional advantage of such types of models is that by integrating information about both peptide and MHC sequences the resulting model shows the higher accuracy per allele in contrast to per-allele models.

Ideas to explore here is pretty much the same as in the **Per-allele** part, but with a nice yet complex addition of MHC amino acid sequences.